In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from dataloading import load_data


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

transform = transforms.Compose([ 
    transforms.Resize((227, 227)),  
    transforms.ToTensor()
])    

data_percentage = 50


train_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

test_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


path:  ../data/vegetable_images/train
needed_length: 7500, expected_length_per_class: 500
length of final dataset: 7500
path:  ../data/vegetable_images/validation
needed_length: 1500, expected_length_per_class: 100
length of final dataset: 1500
path:  ../data/vegetable_images/test
needed_length: 1500, expected_length_per_class: 100
length of final dataset: 1500


In [3]:

concat_dataset = ConcatDataset([train_loader.dataset, valid_loader.dataset, test_loader.dataset])

# Create a new data loader from the concatenated dataset
batch_size = 64  # Set your desired batch size
concat_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True)

In [4]:
total = []
for images, _ in train_loader:
    images, _ = next(iter(concat_loader))
    flattened_image = images.view(images.size(0), images.size(1), -1)
    total.append(flattened_image)
pixel_values = torch.cat(total, dim=0)
pixel_values.shape



torch.Size([7552, 3, 51529])

In [5]:
mean = torch.mean(pixel_values, dim=0)
mean.shape

torch.Size([3, 51529])

In [6]:
mean_per_channel = mean.mean(dim=1)
mean_per_channel

tensor([0.4691, 0.4631, 0.3419])

In [7]:
std = torch.std(pixel_values, dim=0)


In [8]:
std_sq = torch.square(std)
std_mean = std_sq.mean(dim = 1)
std_mean

tensor([0.0539, 0.0516, 0.0573])

In [9]:
std_mean = torch.sqrt(std_mean)
std_mean

tensor([0.2322, 0.2272, 0.2394])